## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np
#  Import and read the charity_data.csv.
import pandas as pd 
app_df = pd.read_csv('./Resources/charity_data.csv')
app_df.head()

2022-11-21 13:58:26.447150: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_df = app_df.drop(['EIN', 'NAME'], axis=1)
app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
app_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
type_counts = app_df['APPLICATION_TYPE'].value_counts()
type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced

Infinity = np.inf
app_type_bins = type_counts.value_counts(bins=[0, 1000, Infinity])
app_type_bins

(-0.001, 1000.0]    12
(1000.0, inf]        5
Name: APPLICATION_TYPE, dtype: int64

In [6]:
app_df['COUNT'] = app_df.groupby(['APPLICATION_TYPE'])['APPLICATION_TYPE'].transform('count')
app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,COUNT
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,528
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,27037
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,1173
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,27037
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,27037


In [7]:
other_df = app_df.loc[app_df['COUNT'] <1000]
saved_df = app_df.loc[app_df['COUNT']>=1000]

In [8]:
# Use the variable name application_types_to_replace
application_types_to_replace = other_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace

T8     737
T7     725
T10    528
T9     156
T13     66
T12     27
T2      16
T25      3
T14      3
T29      2
T15      2
T17      1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
other_df.loc[other_df['APPLICATION_TYPE'] != 'Bart', 'APPLICATION_TYPE'] = 'Other'
new_type = other_df['APPLICATION_TYPE'].value_counts()
new_type

Other    2266
Name: APPLICATION_TYPE, dtype: int64

In [10]:
application_df = pd.concat([saved_df, other_df], axis=0)
application_df = application_df.sort_index(ascending=True)
application_df = application_df.drop(columns=['COUNT'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [11]:
#Check to see if binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [12]:
# You may find it useful to look at classification value counts > 1
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [13]:
# Choose a cutoff value and create a list of classifications to be replaced
class_type_bins = class_counts.value_counts(bins=[0, 1000, Infinity])
class_type_bins

(-0.001, 1000.0]    66
(1000.0, inf]        5
Name: CLASSIFICATION, dtype: int64

In [14]:
application_df['COUNT'] = application_df.groupby(['CLASSIFICATION'])['CLASSIFICATION'].transform('count')
application_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,COUNT
0,Other,Independent,C1000,ProductDev,Association,1,0,N,5000,1,17326
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,6074
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,1918
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,6074
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,17326


In [17]:
other_df = application_df.loc[application_df['COUNT'] < 1000]
saved_df = application_df.loc[application_df['COUNT'] >= 1000]

In [18]:
classifications_to_replace = other_df['CLASSIFICATION'].value_counts()
classifications_to_replace

C7000    777
C1700    287
C4000    194
C5000    116
C1270    114
        ... 
C4120      1
C8210      1
C2561      1
C4500      1
C2150      1
Name: CLASSIFICATION, Length: 66, dtype: int64

In [19]:
other_df.loc[other_df['CLASSIFICATION'] != 'Rita', 'CLASSIFICATION'] = 'Other'
new_class = other_df['CLASSIFICATION'].value_counts()
new_class

Other    2261
Name: CLASSIFICATION, dtype: int64

In [20]:
application_df = pd.concat([saved_df, other_df], axis=0)
application_df = application_df.sort_index(ascending=True)
application_df = application_df.drop(columns=['COUNT'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [21]:
new_class_counts = application_df['CLASSIFICATION'].value_counts()
new_class_counts

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [22]:
# Convert categorical data to numeric with OneHotEncoder
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [23]:
app_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
app_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [24]:
encoder = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list.append
encoder_df = pd.DataFrame(encoder.fit_transform(application_df[app_cat]))

encoder_df.columns = encoder.get_feature_names_out(app_cat)
encoder_df.shape

(34299, 38)

In [25]:
# Merge OneHotEncoder features and drop the originals
application_df = application_df.merge(encoder_df, left_index=True, right_index=True)
application_df = application_df.drop(app_cat, axis='columns')
application_df.shape

(34299, 41)

In [26]:
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [27]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], axis="columns").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=61)

In [28]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3280      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,741
Trainable params: 5,741
Non-trainable params: 0
_________________________________________________________________


2022-11-21 14:06:31.754933: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [30]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
# Train the model
fit_model = model.fit(X_train_scaled, y_train, steps_per_epoch=5, epochs=150)

Epoch 1/150
5/5 [==============================] - 0s 3ms/step - loss: 0.5413 - accuracy: 0.7382
Epoch 2/150
5/5 [==============================] - 0s 3ms/step - loss: 0.5413 - accuracy: 0.7383
Epoch 3/150
5/5 [==============================] - 0s 10ms/step - loss: 0.5412 - accuracy: 0.7385
Epoch 4/150
5/5 [==============================] - 0s 3ms/step - loss: 0.5411 - accuracy: 0.7381
Epoch 5/150
5/5 [==============================] - 0s 3ms/step - loss: 0.5411 - accuracy: 0.7381
Epoch 6/150
5/5 [==============================] - 0s 3ms/step - loss: 0.5411 - accuracy: 0.7384
Epoch 7/150
5/5 [==============================] - 0s 3ms/step - loss: 0.5412 - accuracy: 0.7379
Epoch 8/150
5/5 [==============================] - 0s 3ms/step - loss: 0.5411 - accuracy: 0.7383
Epoch 9/150
5/5 [==============================] - 0s 3ms/step - loss: 0.5411 - accuracy: 0.7387
Epoch 10/150
5/5 [==============================] - 0s 3ms/step - loss: 0.5408 - accuracy: 0.7386
Epoch 11/150
5/5 [==========

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5626 - accuracy: 0.7269 - 246ms/epoch - 919us/step
Loss: 0.5625988841056824, Accuracy: 0.7268804907798767


In [34]:
# Export our model to HDF5 file
model.save('AlphabetSoupCharity_Optimization2.h5')

### Modest increase in accuracy, but nothing to write home about.